# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

[nltk_data] Downloading package punkt to /Users/Shubham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Shubham/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///ETLPrep.db')
df = pd.read_sql_table('ETL_table',engine)
X = df.message
Y = df[df.columns[4:]]
category_names = Y.columns
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: F

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [7]:
print(Y_test)

       related  request  offer  aid_related  medical_help  medical_products  \
14817        1        0      0            0             0                 0   
655          2        0      0            0             0                 0   
23376        1        0      0            1             0                 0   
4930         1        1      0            1             0                 0   
13579        1        0      0            0             0                 0   
...        ...      ...    ...          ...           ...               ...   
584          1        0      0            0             0                 0   
16785        1        0      0            1             0                 0   
25586        1        0      0            0             0                 0   
8632         1        0      0            0             0                 0   
19562        1        0      0            1             0                 1   

       search_and_rescue  security  military  child

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]), 
          '------------------------------------------------------------')

related :
              precision    recall  f1-score   support

           0       0.61      0.37      0.46      1503
           1       0.83      0.93      0.87      4996
           2       0.53      0.38      0.44        55

    accuracy                           0.80      6554
   macro avg       0.66      0.56      0.59      6554
weighted avg       0.78      0.80      0.78      6554
 ------------------------------------------------------------
request :
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5453
           1       0.81      0.34      0.48      1101

    accuracy                           0.88      6554
   macro avg       0.85      0.66      0.71      6554
weighted avg       0.87      0.88      0.85      6554
 ------------------------------------------------------------
offer :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6229
           1       0.30      0.02      0.03       325

    accuracy                           0.95      6554
   macro avg       0.63      0.51      0.50      6554
weighted avg       0.92      0.95      0.93      6554
 ------------------------------------------------------------
direct_report :
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      5266
           1       0.80      0.30      0.43      1288

    accuracy                           0.85      6554
   macro avg       0.82      0.64      0.67      6554
weighted avg       0.84      0.85      0.82      6554
 ------------------------------------------------------------


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7fd76ab30ef0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_feature

In [10]:
parameters = {
    'clf__estimator__n_estimators': [100,150],
    'clf__estimator__min_samples_split': [2,3],
    'tfidf__smooth_idf':[True, False]
    }
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

In [11]:
cv.fit(X_train, Y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True, total= 4.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True, total= 5.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=True, total= 8.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False, total= 4.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False, total= 6.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__smooth_idf=False, total= 5.0min
[CV] clf__estimator

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 123.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'tfidf__smooth_idf': False}


In [15]:
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [18]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i]), '...................................................')

related :
              precision    recall  f1-score   support

           0       0.74      0.29      0.42      1503
           1       0.82      0.97      0.88      4996
           2       0.60      0.38      0.47        55

    accuracy                           0.81      6554
   macro avg       0.72      0.55      0.59      6554
weighted avg       0.80      0.81      0.77      6554
 ...................................................
request :
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5453
           1       0.88      0.41      0.56      1101

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554
 ...................................................
offer :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6393
           1       1.00      0.02      0.05       161

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.96      6554
 ...................................................
missing_people :
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6466
           1       0.75      0.03      0.07        88

    accuracy                           0.99      6554
   macro avg       0.87      0.52      0.53      6554
weighted avg       0.98      0.99      0.98      6554
 ...................................................
refugees :
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6366
           1       0.33      0.01      0.02       188

    accuracy                           0.97      6

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

NameError: name 'model' is not defined

In [21]:
pickle.dump(optimised_model, open('Classifier_Optimized.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.